In [2]:
import torch
from torch import softmax
from torch.nn import CrossEntropyLoss, Softmax
from patbert.common import medical
from os.path import join
data_path = "C:\\Users\\fjn197\\PhD\\projects\\PHAIR\\pipelines\\patbert\\data"
import pickle

c:\Users\fjn197\Miniconda3\envs\patbert\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Example hierarchy

In [81]:
# mask rows which equal another tensor
def mask_not_equal(tensor, other):
    for sample in other:
        tensor = tensor[~(tensor == sample).all(dim=1)]
    return tensor

In [82]:
def get_unique_parents(children, level):
    parent = children.clone()
    parent[:,-level] = 0
    return parent.unique(dim=0)

In [89]:
def select_leaf_nodes(tree):
    all_leafs =[]
    for i in range(tree.shape[1]-1):
        leaf_mask = tree[:,-(i+1)]!=0
        leafs = tree[leaf_mask]
        tree = tree[~leaf_mask]
        all_leafs.append(leafs)
        parent = get_unique_parents(leafs, i+1)
        tree = mask_not_equal(tree, parent)
    all_leafs.append(tree)
    return torch.cat(all_leafs, dim=0)

In [94]:
tree = torch.tensor([[1, 1, 0], [1,1,1], [1,1,2], [1,2,0],[1,0,0],[2,1,0],[2,2,0],[3,0,0]])
select_leaf_nodes(tree)

tensor([[1, 1, 1],
        [1, 1, 2],
        [1, 2, 0],
        [2, 1, 0],
        [2, 2, 0],
        [3, 0, 0]])

In [17]:

leafs_0 = tree[tree[:,-1]!=0]
temp = leafs_0
temp[:,-1] = 0
temp = temp.unique(dim=0)
#remove temp from tree
tree = mask_not_equal(tree, temp)
print(tree)
# remove lowest leafs from tree
tree = tree[tree[:,-1]==0]
leafs_1 = tree[tree[:,-1]!=0]
# remove lowest leafs from tree
tree


tensor([[1, 1, 1],
        [1, 1, 2],
        [1, 2, 0],
        [2, 1, 0],
        [2, 2, 0],
        [3, 0, 0]])


tensor([[1, 2, 0],
        [2, 1, 0],
        [2, 2, 0],
        [3, 0, 0]])

In [2]:
main_vocab = torch.load(join(data_path,"tokenized\\mimic3\\plain\\vocabulary.pt"))
atc_codes = medical.MedicalCodes().get_atc()
icd_codes = medical.MedicalCodes().get_icd()
codes = []
codes.append('[PAD]')
codes = codes + atc_codes[:5] + icd_codes[:5]

In [3]:
codes

['[PAD]',
 'MA01',
 'MA01A',
 'MA01AA',
 'MA01AA01',
 'MA01AA02',
 'DA00',
 'DA000',
 'DA001',
 'DA009',
 'DA01']

In [4]:
main_vocab = {}
for random_code in codes:
    main_vocab[random_code] = len(main_vocab)
vocab_ls = medical.SKSVocabConstructor(main_vocab)()

In [5]:
pad = []
ma01 = []
ma01aa02  = []
da009 = []
for vocab in vocab_ls:
    pad.append(vocab['[PAD]'])
    da009.append(vocab['DA009'])
    ma01.append(vocab['MA01'])
    ma01aa02.append(vocab['MA01AA02'])
print(pad)
print(da009)
print(ma01)
print(ma01aa02)


[0, 0, 0, 0, 0, 0]
[1, 1, 2, 11, 0, 0]
[2, 1, 3, 0, 0, 0]
[2, 1, 3, 12, 12, 4]


In [192]:
for vocab in vocab_ls:
    print(vocab['DA009'])

1
1
2
11
0
0


In [49]:
vocab = {
    'D':1,
    'M':2,
    'DA00':10,
    'DA01':11,
    'DB99':13,
    'MA00':20,
    'DA000':100,
    'DA001':101,
    'MA000':200,
    'MA001':201,}

In [6]:
def get_ex_pred(ids):
    ex_pred = torch.rand(204)
    for id in ids:
        ex_pred[id] = 100
    return softmax(ex_pred)
def get_targets(target_id):
    targets_ls = []
    while target_id>0:
        targets = torch.zeros(204)
        targets[target_id] = 1
        targets_ls.append(targets)
        target_id = target_id//10
    return targets_ls


In [7]:
# import cross entropy loss
ce = CrossEntropyLoss()
def f_loss(pred, targets):
    loss = 0 
    for i, target in enumerate(targets):
        loss+= 1/(10**i)*float(ce(pred.unsqueeze(0), target.unsqueeze(0)) )
    return loss

In [8]:
target_id = 201
targets = get_targets(target_id)

In [3]:
target = torch.zeros(10)
target[0] = 1

In [24]:
torch.tensor(logits, dtype=torch.float32)

tensor([ 0., 10., 10.,  0.,  5.,  5.])

In [43]:
tokens = ['A', 'B', 'Aa', 'Ab', 'Ba', 'Bb']
logits = [0, 100, 100, 0, 0, 100]
level0_probs = Softmax(dim=0)(torch.tensor(logits[:2], dtype=torch.float32))
level1_probs_A = Softmax(dim=0)(torch.tensor(logits[2:4], dtype=torch.float32))
level1_probs_B = Softmax(dim=0)(torch.tensor(logits[4:], dtype=torch.float32))
print(level0_probs)
print(level1_probs_A)
print(level1_probs_B)

tensor([3.7835e-44, 1.0000e+00])
tensor([1.0000e+00, 3.7835e-44])
tensor([3.7835e-44, 1.0000e+00])


RuntimeError: 0D or 1D target tensor expected, multi-target not supported